# -*- coding: utf-8 -*-

"""
Exemplos de validação

@author(es): Daniel, Francisco, Anderson, Leomar, Victor, Leonardo, Regiane.
@GrupoPesquisa: PROTEC
@LinhadePesquisa: GI-UFBA

Exemplo retirado de  Sanjari (2013, p.12) que trata sobre como calcular a pressão de vapor de substâncias puras em função da temperatura reduzida. A pressão crítica é apresentada usando o algoritmo de Marquardt-Levenberg que minimiza a soma das diferenças quadráticas entre os valores dos valores observados e previstos das variável dependente:

Exemplo básico de uso do MT_PEU, onde são apresentadas as configurações mínimas para o funcionamento do software.

SUMÁRIO:

I - INCLUSÃO DAS BIBLIOTECAS

II - CRIAÇÃO DO MODELO

III - INICIALIZAÇÃO DA CLASSE

IV - INCLUSÃO DE DADOS

V - OTIMIZAÇÃO

VI - INCERTEZA

VII - PREDIÇÃO E ANALISE DE RESIDUOS

VIII - GERAÇÃO DE GRÁFICOS E RELATÓRIO

"""

# Importação de pacotes

importação de bibliotecas (pacotes) necessárias para execução do código.

* MT_PEU: biblioteca que contém as principais funcionalidades da ferramenta
* numpy: biblioteca para computação científica -> será necessária somente a função exp (exponencial)

In [1]:
from matplotlib import use
use('Agg')

from MT_PEU import EstimacaoNaoLinear
from numpy import exp

# CRIAÇÃO DO MODELO

In [2]:
u"""
O modelo é definido na forma de uma subrotina ((def) do python) e representa a equação abaixo,onde: 
y é a pressão de vapor, T é a temperatura, e por fim A, B e C são os parâmetros a serem estimados.

"""
def Modelo(param,x,args):

    T = x[:,0:1] # primeira coluna do vetor de dado

    A = param[0] # primeiro parâmetro
    B  = param[1] # segundo parâmetro
    C = param[2] # terceiro parÂmetro
    
    Pvp = exp(A-(B/(T+C))) # cálculo de Pvp - vetorizado

    return Pvp

# ==========================
 IV - INICIALIZAÇÃO DA CLASSE
# ==========================

In [3]:
u"""

Nessa etapa é inicializada a classe que realiza a estimação. Por padrão, informações como os simbolos das variáveis,
são obrigatorias e passadas nesta etapa.  

"""

#Cria o objeto que realiza a estimação

Estime = EstimacaoNaoLinear(Modelo, simbolos_x=[r'T'], simbolos_y=[r'Pvp'], simbolos_param=['A','B','C'])

# ===============================================================

IV - INCLUSÃO DE DADOS

# ===============================================================


In [4]:
u"""

Os dados experimentais da variável dependente (Pvp) e da variável independente (Temperatura)
são disponibilizados em Zaitseva, Zaitsau, Varfolomeev e Verevkin (2019, p.230) apresentados 
abaixo na forma de listas:

"""

Pvp = [2.93,3.21,3.49,4.22,5.60,7.31,9.12,13.07,14.98,17.63,18.02,22.08,26.95,34.61,
       40.93,50.17,63.36,78.93,93.65,115.11,140.27,171.89,208.00]


temperatura = [297.1,298.2,299.3,301.2,304.2,307.2,310.2,314.1,316.2,317.8,318.2,320.2,
               323.1,326.2,329.1,331.2,334.2,337.1,340.2,343.2,346.2,349.1,352.2]

In [5]:
u"""

Como entrada obrigatória, a plataforma MT_PEU necessita da incerteza dos dados experimentais (ux1, ux2, uy1).
Neste exemplo, foram adotados o valor 1 para as incertezas.

"""

# lista com incertezas para a temperatura
uxtemperatura = [1]*23
# lista com incertezas para Pvp
uPvp = [1]*23

In [6]:
u"""

Inclusão de dados experimentais na estimação:
Inclui os dados experimentais nesse objeto (setDados), onde a opção 0 é para a grandeza dependente,
e a opção 1 é para a grandeza independente.

"""
# grandezas dependentes. [Opção 0 é para grandeza dependente]
Estime.setDados(0,(temperatura,uxtemperatura))
# grandezas independentes [Opção 1 é para grandeza independente]
Estime.setDados(1,(Pvp,uPvp))

In [7]:

u"""

Define que os dados experimentais previamente inseridos serão utilizados como um conjunto de dados para o qual os 
parâmetros serão estimados:

"""

Estime.setConjunto(tipo='estimacao')

# ==============
 V - OTIMIZAÇÃO
# ==============


In [8]:
u"""

A otimização será realizada utilizando o algoritmo default (Nelder-Mead), se faz necessário informar a estimativa inicial.

"""
Estime.otimiza(estimativa_inicial= [1, 1.5, 0.009])

# =============
VI - INCERTEZA
# =============

In [9]:
u"""
 "Associada a toda medida existe uma incerteza." 
 Este método calcula a incerteza associadas aos parâmetros (neste exemplo A, B e C). 
 
"""

Estime.incertezaParametros()

/home/giufba-admin/Documentos/C.I 2019.2/MT_PEU-Teste/MT_PEU.py:1862: UserWarning: A região de abrangência avaliada pelo método da verossimilhança não contém pontos. Reveja os parâmetros do algoritmo utilizado.
  warn('A região de abrangência avaliada pelo método da verossimilhança não contém pontos. Reveja os parâmetros do algoritmo utilizado.',UserWarning)


# =================================================================================
 VII - PREDIÇÃO E ANALISE DE RESIDUOS
# =================================================================================

In [10]:
u"""
 No método predição, é feita a avaliação da grandeza dependente com base nos parametros fornecidos. 
 A covariância é avaliada, e consequentemente a eficiencia do modelo. 
 Em analise de residuos é possível vericar possíveis relações de dependencia e/ou tendencia entre as variaveis. 
 Testes estatisticos como o de homocedasticidade, chi quadrado, dentre outros são realizados
 nesta etapa. A analise de residuos é feita prioritariamente com os dados de validação.

"""

Estime.predicao()
Estime.analiseResiduos()

# ===================================
 VIII - GERAÇÃO DE GRÁFICOS E RELATÓRIO
# ===================================

In [11]:
u"""
 Nesta etapa ocorre a geração dos dados de saída do programa : relátorios e gráficos. 
 Os gráficos são gerados de acordo com as etapas que foram realizadas. No relátorio contém informações a respeito
 dos testes estatisticos, função objetivo, matriz de covariância, status da otimização, dentre outros.

"""
etapas = ['otimizacao', 'grandezas-entrada', 'predicao', 'grandezas-calculadas', 'analiseResiduos', 'regiaoAbrangencia']
Estime.graficos(etapas)
Estime.relatorio()

/home/giufba-admin/Documentos/C.I 2019.2/MT_PEU-Teste/MT_PEU.py:2084: UserWarning: Os gráficos de otimizacao não puderam ser criados, o algoritmo de otimização utilizado não possui gráficos de desempenho OU o método otimizacao não foi executado.
  warn('Os gráficos de otimizacao não puderam ser criados, o algoritmo de otimização utilizado não possui gráficos de desempenho OU o método otimizacao não foi executado.',UserWarning)


In [ ]:
u"""

Referências: 

SANJARI, Ehsan. A new simple method for accurate calculation of saturated vapor pressure. Elsevier. 
p. 12-16. mar. 2013.

ZAITSEVA, Ksenia V.; ZAITSAU, Dzmitry H.; VARFOLOMEEV, Mikhail A.. 
Vapour pressures and enthalpies of vaporisation of alkyl formamides. Elsevier. Alemanha, p. 228-238. maio 2019.

Avaliação de dados de medição — Guia para a expressão de incerteza de medição 
http://www.inmetro.gov.br/noticias/conteudo/iso_gum_versao_site.pdf 


"""